# Building a Spam Filter with Naive Bayes
In this project, our goal is to use the Naive Bayes algorithm to build a spam filter for SMS messages.

To classify messages as spam or non-spam, the computer:
- Learns how humans classify messages.
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

Our first task will be to "teach" the computer how to classify messages. We'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

We'll start by reading in the dataset.

*Note that due to the nature of spam messages, the dataset contains content that may be offensive to some users.*

In [1]:
import warnings

warnings.simplefilter(action='ignore')
import numpy as np
import pandas as pd
import math
import re
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
df.shape

(5572, 2)

In [3]:
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Our dataset contains 13.4% spam messages and 86.6% ham (non-spam) messages.

### Training and Test Set
Before creating our spam filter, we need to design a way to test its efficacy. We'll do this by testing how good it is with classifying new messages. To conduct this test, we need to split the dataset into two categories:
- A training set, which we'll use to "train" the computer to classify messages.
- A test set, which we'll use to test how good the spam filter is with classifying new messages.

Our train set will contain 80% of the dataset and the remainder 20% will be used for testing. The dataset has 5,572 messages, which means that:
- The train set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

After training and testing, we'll be able to compare the algorithm classification with the classification done by a human, to confirm the efficacy of the spam filter. The success threshold for our spam filter is 80%, meaning that a successful spam filter will have an accuracy greater than 80%.

We'll begin by creating a train and a test set.

In [5]:
# Randomize the dataset
df_random = df.sample(frac=1, random_state=1)

# Calculate index for split
train_index = round(len(df_random) * 0.8)

# Train-Test Split
train_set = df_random[:train_index].reset_index(drop=True)
test_set = df_random[train_index:].reset_index(drop=True)

Let's find the percentage of spam and ham in both the training and the test set.

In [6]:
train_set['Label'].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [7]:
test_set['Label'].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

These value counts are identical to those in the original dataset.

### Letter Case and Punctuation
Now that we've split our dataset into a train and test set. The next big step is to use the training set to teach the algorithm to classify new messages.

In order to classify messages using the Naive Bayes algorithm, we'll use the following formula:
- P(Spam|w1,w2,...,wn) ∝ P(Spam) * n∏i=1(P(wi|Spam))
- P(Ham|w1,w2,...,wn) ∝ P(Ham) * n∏i=1(P(wi|Ham))

To calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:
- P(wi|Spam) = Nwi|Spam + α / NSpam + α * NVocabulary
- P(wi|Ham) = Nwi|Ham + α / NHam + α * NVocabulary

Let's summarize what the terms in the equations above mean:
- Nwi|Spam = the number of times the word wi occurs in spam messages
- Nwi|Spam^C/Ham = the number of times the word wi occurs in non-spam messages
- NSpam = total number of words in spam messages
- NSpam^C/Ham = total number of words in non-spam messages
- NVocabulary = total number of words in the vocabulary
- α = 1 (α is a smoothing parameter)

To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. We need to use one-hot encoding (or dummy variables) to convert the categorical/ text values in the SMS column into integer values to make them easy for the algorithm to interpret. The categorical values become columns in the dataset used to count how many times the value occurs in an SMS record.

All words in the vocabulary (i.e. categorical values in the columns) are in lower case. Upper case is ignored and all upper case values will be counted with their lower case counterparts. Punctuations are ignored.

We'll begin the data cleaning process by removing the punctuation and bringing all the words to lower case.

In [8]:
train_set['SMS'] = train_set['SMS'].str.replace('\W', ' ', regex=True).str.lower()
train_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [9]:
test_set['SMS'] = test_set['SMS'].str.replace('\W', ' ', regex=True).str.lower()
test_set.head()

,Label,SMS
0,ham,later i guess i needa do mcat study too
1,ham,but i haf enuff space got like 4 mb
2,spam,had your mobile 10 mths update to latest oran...
3,ham,all sounds good fingers makes it difficult ...
4,ham,all done all handed in don t know if mega sh...


### Creating the Vocabulary
We'll now create a list that contains all the unique words that occur in the messages of our training set (the vocabulary).

In [10]:
train_set['SMS'] = train_set['SMS'].str.split()
train_set.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [11]:
vocabulary = list()
for list_item in train_set['SMS']:
    for item in list_item:
        vocabulary.append(item)

len(vocabulary)

72427

In [12]:
vocabulary = list(set(vocabulary))
len(vocabulary)

7783

### The Final Training Set
We'll create a dictionary to count the occurrences of unique values in the vocabulary across the train set, then convert the dictionary to a dataframe.

We'll start by initializing a dictionary where each key is a unique word (a string) from the vocabulary, and each value is a list of the length of training set, and where each element in the list is a 0.

We'll then loop over the SMS column in the train set and using a nested loop, we'll increment the word count for each word in each sms.

In [13]:
word_counts_per_sms = {unique_word: [0] * len(train_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

We'll now transform the dictionary into a dataframe and join that dataframe to the train set.

In [14]:
word_count = pd.DataFrame(word_counts_per_sms)
train_df = pd.concat([train_set, word_count], axis=1)
train_df.head()

,Label,SMS,08709501522,received,book,shorter,adult,eshxxxxxxxxxxx,odalebeku,wales,...,overa,audrey,regalportfolio,minute,food,countin,colin,09064015307,peace,hmv
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculating Constants First
Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. The Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:
- P(Spam|w1,w2,...,wn) ∝ P(Spam) * n∏i=1(P(wi|Spam))
- P(Ham|w1,w2,...,wn) ∝ P(Ham) * n∏i=1(P(wi|Ham))

To calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:
- P(wi|Spam) = Nwi|Spam + α / NSpam + α * NVocabulary
- P(wi|Ham) = Nwi|Ham + α / NHam + α * NVocabulary

We'll start by calculating P(Spam), P(Ham), NSpam, NHam, NVocabulary.
NSpam is the total number of words in all spam messages while NHam is the total number of words in all Ham (non-spam) messages. We'll also use Laplace smoothing and set α = 1.

In [15]:
alpha = 1

p_ham = train_df['Label'].value_counts(normalize=True).loc['ham']
p_spam = train_df['Label'].value_counts(normalize=True).loc['spam']
print(p_ham, p_spam)

0.8654104979811574 0.13458950201884254


In [16]:
ham_msg = train_df.loc[train_df['Label'] == 'ham']
spam_msg = train_df.loc[train_df['Label'] == 'spam']

In [17]:
n_ham = 0
for list_item in ham_msg['SMS']:
    for item in list_item:
        n_ham += 1

n_spam = 0
for list_item in spam_msg['SMS']:
    for item in list_item:
        n_spam += 1

n_vocabulary = len(vocabulary)
print(n_ham, n_spam, n_vocabulary)

57237 15190 7783


### Calculating Parameters
The values P(Spam), P(Ham), NSpam, NHam, NVocabulary are constants in our equations for every new message. However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. The probability values that P(wi|Spam) and P(wi|Ham) will take are called parameters.

We'll initialize two dictionaries, where each key-value pair is a unique word (from our vocabulary) represented as a string, and the value is 0. We'll need one dictionary to store the parameters for P(wi|Spam), and the other for P(wi|Ham).

In [18]:
p_ham_words = {unique_word: 0 for unique_word in vocabulary}
p_spam_words = {unique_word: 0 for unique_word in vocabulary}

Let's check one of the dictionaries.

In [19]:
for idx, (k, v) in enumerate(p_ham_words.items()):
    if idx == 5: break
    print(k, v)

08709501522 0
received 0
book 0
shorter 0
adult 0


Now we'll calculate the parameters for each word.

In [20]:
for word in vocabulary:
    n_word_given_spam = spam_msg[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + (alpha * n_vocabulary))
    p_spam_words[word] = p_word_given_spam

    n_word_given_ham = ham_msg[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    p_ham_words[word] = p_word_given_ham


If we check the updated dictionaries, we can see the values have updated to reflect the probability of each word given spam or non-spam.

In [21]:
for idx, (k, v) in enumerate(p_ham_words.items()):
    if idx == 5: break
    print(k, v)

08709501522 1.537988311288834e-05
received 4.6139649338665025e-05
book 0.00019993848046754843
shorter 4.6139649338665025e-05
adult 3.075976622577668e-05


### Classifying A New Message
Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:
- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [22]:
def classify(message):

    # Clean message string
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    # calculate probability given message
    for word in message:
        if word in p_ham_words:
            p_ham_given_message *= p_ham_words[word]
        if word in p_spam_words:
            p_spam_given_message *= p_spam_words[word]

    # print probability value for message
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    # Classify message based on probability values
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities, have a human classify this!')

Let's test the function.

In [23]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [24]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Measuring the Spam Filter's Accuracy
We've created a spam filter classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages. The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

Let's update the classify() function to return the labels instead of printing them.

In [25]:
def classify_test_set(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_ham_words:
            p_ham_given_message *= p_ham_words[word]
        if word in p_spam_words:
            p_spam_given_message *= p_spam_words[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we've updated our function, let's use it to create a new column in our test set.

In [26]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,later i guess i needa do mcat study too,ham
1,ham,but i haf enuff space got like 4 mb,ham
2,spam,had your mobile 10 mths update to latest oran...,spam
3,ham,all sounds good fingers makes it difficult ...,ham
4,ham,all done all handed in don t know if mega sh...,ham


Let's compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:
Accuracy = number of correctly classified messages / total number of classified messages

In [27]:
correct = 0
total = len(test_set)
for index, row in test_set.iterrows():
    if row['Label'] == row['predicted']:
        correct += 1

accuracy = correct / total
accuracy

0.9874326750448833

Our accuracy metric shows that the algorithm is 99% accurate, which is well above our goal accuracy.

### Next Steps
Here's a few next steps:
- Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
- Make the filtering process more complex by making the algorithm sensitive to letter case.